# Sample RAG Agent Walkthrough

This notebook will walk users through setting up a sample RAG Agent that uses the Hugging Face 'rag-mini-wikipedia' dataset (https://huggingface.co/datasets/rag-datasets/rag-mini-wikipedia)

This agent utilizes Bedrock Knowledge Base for the RAG capability.

## Pre-requisites

This notebook assumes that you have gone through the notebook environment setup in the agents_catalog/0-Notebook-environment/ folder

#### Ensure the latest version of boto3 is shown below

##### If not then run through setup_environment.ipynb in the agents_catalog/0-Notebook-environment/ folder

In [ ]:
!pip freeze | grep boto3

#### Load in environment variables to notebook

In [ ]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r account_id
%store -r region

# Retrieve model lists
%store -r agent_foundation_model

#### Retrieve imports environment variable and bring libraries into notebook

In [ ]:
%run $IMPORTS_PATH

# Data Preparation

### Retrieve Hugging Face 'rag-mini-wikipedia'

In [ ]:
# Retrieves dataset corpus using Datasets Python library

ds_corpus = load_dataset("rag-datasets/rag-mini-wikipedia", "text-corpus")

### Write text corpus to file and upload to Amazon S3 to use as data source for knowledge base

In [ ]:
# Write whole corpus to a .txt file

with open('mini_wiki.txt', 'w') as f:
    f.write(str(ds_corpus['passages']['passage']))

print("You can now view the whole Wikipedia corpus in mini_wiki.txt")

In [ ]:
# Create Amazon S3 bucket and upload .txt. file to Amazon S3 bucket

s3_client = boto3.client('s3')

wiki_bucket_name = f"rag-mini-wiki-{account_id}-{uuid.uuid4().hex[:6]}"

if region == 'us-east-1':
    s3_client.create_bucket(
        Bucket=wiki_bucket_name
    )
else:
    s3_client.create_bucket(
        Bucket=wiki_bucket_name,
        CreateBucketConfiguration={
            'LocationConstraint': region
        }
    )

%store wiki_bucket_name

print("Created bucket with name '{}' in region '{}'".format(wiki_bucket_name, region))

In [ ]:
# Place .txt corpus in S3 bucket

s3_client.upload_file('mini_wiki.txt', wiki_bucket_name, 'mini_wiki.txt')

print("Uploaded corpus to '{}'".format(wiki_bucket_name))

### Create Bedrock Knowledge Base
#### Follow the steps below to create a Bedrock Knowledge Base in the AWS Console manually

Step 1: Navigate to the 'Amazon Bedrock' service in the AWS Console and navigate to the 'Knowledge Bases' section

Step 2: Click 'Create' and select 'Knowledge Base with vector store'

Step 3: Name the Knowledge Base 'mini-wiki-kb' and select the Amazon S3 data source radio button

Step 4: Name the data source 'mini-wiki-data' and select the S3 bucket file 'mini_wiki.txt' that was uploaded, 
        e.x. s3://rag-mini-wikipedia-data-XXXXXXXXXXXX/mini_wiki.txt

Step 5: Use the default parsing and default chunking options

Step 6: Select the 'Titan Text Embeddings V2' embedding model and create an Amazon OpenSearch Serverless vector store with the quick create option

Step 7: Now create the knowledge base (this process may take several minutes)

Step 8: Manually sync the data source with the knowledge base by clicking on the data source and selecting 'Sync' and wait for the process to finish before proceeding to the next step

In [ ]:
# Fetch knowledge base ID

bedrock_agent_client = boto3.client("bedrock-agent", region)

# Call the list_knowledge_bases method
response = bedrock_agent_client.list_knowledge_bases()
wiki_kb_id = None

# Iterate through knowledge bases and find needed one
if 'knowledgeBaseSummaries' in response:
    for kb in response['knowledgeBaseSummaries']:
        if 'mini-wiki-kb' in kb['name']:
            wiki_kb_id = kb['knowledgeBaseId']

%store wiki_kb_id

wiki_kb_id

# Create RAG Agent

In [ ]:
agent_name = 'sample-rag-agent'
agent_description = "RAG agent to run against the Hugging Face 'rag-mini-wikipedia' dataset"
agent_instruction = """Use the associated knowledge base to answer questions."""

In [ ]:
agents = AgentsForAmazonBedrock()

rag_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=False,
    verbose=False
)

rag_agent

In [ ]:
rag_agent_id = rag_agent[0]
rag_agent_arn = f"arn:aws:bedrock:{region}:{account_id}:agent/{rag_agent_id}"

rag_agent_id, rag_agent_arn

In [ ]:
agents.associate_kb_with_agent(
    rag_agent_id,
    "Hugging Face 'rag-mini-wikipedia' dataset", 
    wiki_kb_id
)

### Test RAG Agent

#### Invoke Sample RAG Agent Test Alias to see that it answers question properly

In [ ]:
# Ask example question to agent

bedrock_agent_runtime_client = boto3.client("bedrock-agent-runtime", region)

session_id:str = str(uuid.uuid1())

test_query = "Who suggested Lincoln grow a beard?"
response = bedrock_agent_runtime_client.invoke_agent(
      inputText=test_query,
      agentId=rag_agent_id,
      agentAliasId="TSTALIASID", 
      sessionId=session_id,
      enableTrace=True, 
      endSession=False,
      sessionState={}
)

print("Request sent to Agent")
print("====================")
print("Agent processing query now")
print("====================")

# Initialize an empty string to store the answer
answer = ""

# Iterate through the event stream
for event in response['completion']:
    # Check if the event is a 'chunk' event
    if 'chunk' in event:
        chunk_obj = event['chunk']
        if 'bytes' in chunk_obj:
            # Decode the bytes and append to the answer
            chunk_data = chunk_obj['bytes'].decode('utf-8')
            answer += chunk_data

# Now 'answer' contains the full response from the agent
print("Agent Answer: {}".format(answer))
print("====================")

#### Prepare agent and create alias

In [ ]:
rag_agent_alias_id, rag_agent_alias_arn = agents.create_agent_alias(
    rag_agent[0], 'v1'
)

%store rag_agent_alias_arn
rag_agent_alias_id, rag_agent_alias_arn

# Create question and answer file
Note: The 'question_and_answers.json' file compiles a list of questions and answers from the dataset, you can try asking the RAG agent some of these questions!

In [ ]:
# Create input json data file for evaluation framework and place so it can be run by evaluation framework by user

ds_qa = load_dataset("rag-datasets/rag-mini-wikipedia", "question-answer")

qa_data = []

# Iterate through all elements in dataset
for index, data in enumerate(ds_qa['test']):
    # Append qa to list
    qa_data.append(data)

# Save to JSON file
with open('question_and_answers.json', 'w', encoding='utf-8') as f:
    json.dump(qa_data, f, indent=4, ensure_ascii=False)